In [1]:
import pyspark

In [9]:
from pyspark.sql.types import StructField, StringType, StringType

In [18]:
from pyspark.sql.functions import *

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [26]:
df = spark.read.csv('case.csv', header=True, inferSchema=True)

Read the case, department, and source data into their own spark dataframes.

In [10]:
df_dept = spark.read.csv('dept.csv', header=True, inferSchema=True)
df_source = spark.read.csv('source.csv', header=True, inferSchema=True)

In [27]:
#event table
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [73]:
df.count()

841704

In [11]:
#dimension table
df_dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [12]:
#dimension table
df_source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [13]:
df.write.json("case.json", mode='overwrite')

In [16]:
df.write.json("dept.json", mode='overwrite')
df.write.json("source.json", mode='overwrite')

In [14]:
blob = spark.read.format('json').load('case.json/')

In [15]:
blob.printSchema()

root
 |-- SLA_days: double (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_id: long (nullable = true)
 |-- case_late: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- council_district: long (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- request_address: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- source_id: string (nullable = true)



Inspect your folder structure. What do you notice?

    Those JSONs are a directory rather than one JSON file. I tried reading from it and it came out the way I wrote. So I'll jsut have to remember that sparkDF.write.json makes a directory.

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [28]:
#The withColumn makes a new column. Since case_closed existed before we are mutating it. It's now a boolean value where
#before it was a string. 
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "yes"')
)

In [29]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [25]:
# fmt = "M/d/yy H:mm"

# df = (
#     df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
#     .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
#     .withColumn("SLA_due_date", to_timestamp("SLA_opened_date", fmt))
# )

In [32]:
df = df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))

In [33]:
df = df.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))

In [37]:
df = df.withColumn("SLA_due_date", to_timestamp("SLA_due_date", fmt))

In [38]:
#df = df.withColumn("case_due_date", to_timestamp("case_due_date", fmt))
df.select("case_opened_date", "case_closed_date", "SLA_due_date").show(5)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|       SLA_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



How old is the latest (in terms of days past SLA) currently open issue? 

In [45]:
df = df.withColumn(
        "case_age_past_due", datediff(current_timestamp(), "SLA_due_date"))

In [48]:
open_cases = df.filter(df.case_closed == 'false')

In [49]:
open_cases.sort(open_cases.case_age_past_due.desc()).show(1, vertical=True)
#mpg.sort(mpg.hwy).show(8)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | false                
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_age_past_due    | 1066                 
only showing top 1 row



How long has the oldest (in terms of days since opened) currently opened issue been open?

In [51]:
df_with_case_age = df.filter(df.case_closed == 'false').withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date"))

In [ ]:
open_cases = open_cases.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))

In [75]:
open_cases.sort(open_cases.case_age.desc()).first().case_opened_date

datetime.datetime(2017, 1, 1, 13, 48)

In [58]:
open_cases.sort(open_cases.case_age.desc()).show(1,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | false                
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_age_past_due    | 1066                 
 case_age             | 1082                 
only showing top 1 row



How many Stray Animal cases are there?

In [62]:
df.filter(df.service_request_type == "Stray Animal").count()

26760

In [60]:
df.select("service_request_type").distinct().show(truncate=te=False)

+--------------------------------------+
|service_request_type                  |
+--------------------------------------+
|Minimum Housing-Owner Occupied        |
|Tree Removal                          |
|Service Information                   |
|Sign Maintenance                      |
|Park Building Maint Invest            |
|Brush Property Damage                 |
|Graffiti: Private Property (Corridors)|
|Traffic Sign Graffiti                 |
|License Renewal Invoice               |
|Used/Scrap Tire Facility Registration |
|Guardrail- New Request                |
|Markings Installation SMO (NEW)       |
|CCO_Request for Research/Information_1|
|Sewer Line Broken                     |
|Zoning: Multi-Family In Single        |
|Engineering Investigation             |
|Zoning: Setbacks                      |
|Traffic Sign Faded                    |
|Permits, Fences                       |
|Certificates of Occupancy             |
+--------------------------------------+
only showing top

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [63]:
df.filter(df.dept_division == "Field Operations").where(df["service_request_type"] != "Officer Standby").count()

113902

Convert the council_district column to a string column.

In [30]:
#Made a new column that takes the place of the old one. 
#.withcolumn looks like it takes two parameters: 1 is the name of the new column
#                                               2 is the terms of what this new column will be
df = df.withColumn('council_district', col('council_district').cast("string"))
#df = df.withColumn("council_district", col("council_district").cast("string"))

Extract the year from the case_closed_date column.

In [65]:
df = df.withColumn("year_closed", year("case_closed_date"))

In [68]:
df.show(2,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age_past_due    | -282                 
 year_closed          | 2018                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05

Convert num_days_late from days to hours in new columns num_hours_late.

In [69]:
df = df.withColumn(
    "num_hours_late", expr("case_age_past_due / 7 AS num_hours_late")
)

In [70]:
df.select(df.num_hours_late).show(5)

+-------------------+
|     num_hours_late|
+-------------------+
|-40.285714285714285|
| 101.85714285714286|
| 101.85714285714286|
| 100.14285714285714|
| 102.42857142857143|
+-------------------+
only showing top 5 rows



In [72]:
type(df.select(max('case_closed_date')).head())

pyspark.sql.types.Row

Join the case data with the source and department data.

Are there any cases that do not have a request source?

What are the top 10 service request types in terms of number of requests?

What are the top 10 service request types in terms of average days late?

Does number of days late depend on department?

How do number of days late depend on department and request type?